# Book Writing Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/book_writing_agents.ipynb)

## Dependencies

In [37]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install duckduckgo-search > /dev/null

## Tools

In [50]:
import os
import time
import urllib
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from duckduckgo_search import DDGS
from praisonai_tools import BaseTool, SerperDevTool, FileReadTool

class ScrapeWebsiteTool(BaseTool):
    name: str = "WebContentReaderTool"
    description: str = "Fetches and reads the main text content from a specified webpage URL."

    def _run(self, url: str) -> str:
        """Reads the content of a webpage and returns up to 5000 characters of text."""
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract and clean the text content
            text_content = soup.get_text(separator="\n", strip=True)
            return text_content[:5000]  # Limit content to 5000 characters for brevity
        except requests.exceptions.RequestException as e:
            return f"Failed to retrieve content from {url}: {e}"


## YAML Prompt

In [51]:
agent_yaml = """
framework: "crewai"
topic: "Automated Book Writing on The Metaphorical Tree of Life"
roles:
  researcher:
    role: "Research Agent"
    backstory: |
      A seasoned researcher, known for gathering the best sources and understanding the key elements of any topic.
      Your skill lies in collecting relevant information to support both the book outline and chapter writing process.
    goal: "Gather comprehensive information to support book outline creation and chapter content."
    tasks:
      research_topic:
        description: |
          Research the topic of "The Metaphorical Tree of Life" to gather the most important information that will
          be useful in creating a well-organized book outline. Focus on high-quality, reliable sources.

          The goal of this book is to explore the metaphor of the Tree of Life, delving into its symbolism across cultures,
          its representation of growth, interconnectedness, and resilience, and how it relates to the human experience
          and natural world. The book aims to provide readers with insights into the Tree of Life as a powerful metaphor
          for understanding life's complexities, personal development, and interconnectedness.
        expected_output: "A set of key points and important information about 'The Metaphorical Tree of Life' to be used in creating the outline."
      research_chapter:
        description: |
          Research the provided chapter topic, title, and outline to gather additional content that will be helpful in writing the chapter.
          Ensure you focus on reliable, high-quality sources of information.

          The goal of this book is to explore the metaphor of the Tree of Life, delving into its symbolism across cultures,
          its representation of growth, interconnectedness, and resilience, and how it relates to the human experience
          and natural world. The book aims to provide readers with insights into the Tree of Life as a powerful metaphor
          for understanding life's complexities, personal development, and interconnectedness.

          Here is the outline description for the chapter:

          **"An exploration of the roots as the foundation of the Tree of Life, symbolizing the origins of existence, connection to heritage, and the grounding principles that shape individual identity. This chapter delves into how roots anchor the tree and sustain growth, representing the importance of understanding one’s own beginnings and cultural heritage."**

          When researching, consider the following key points:
          - Gather enough information to write a 3,000-word chapter.
          - Ensure the chapter content aligns with the rest of the book.

          Here is the outline of the entire book:

          - **Chapter 1: Roots and Beginnings**
            An exploration of the roots as the foundation of the Tree of Life, symbolizing origins, connection to heritage, and grounding principles.

          - **Chapter 2: The Trunk – Stability and Growth**
            Examines the trunk as the support system of the tree, representing resilience, stability, and the journey of personal growth.

          - **Chapter 3: Branches of Knowledge**
            Discusses branches as pathways of exploration, illustrating how experiences, learning, and relationships expand one’s reach.

          - **Chapter 4: Leaves – Diversity and Change**
            Leaves symbolize individuality, diversity, and change, representing the unique qualities each person contributes to the world.

          - **Chapter 5: Fruits – Achievement and Legacy**
            Reflects on fruits as manifestations of effort and achievements, the legacy one leaves behind.

          - **Chapter 6: Seasons of Life**
            Analyzes the changing seasons as phases of life, symbolizing the cyclical nature of growth, dormancy, and renewal.
        expected_output: "A set of additional insights and information to be used in writing the chapter."
    tools:
      - "SerperDevTool"

  outliner:
    role: "Book Outlining Agent"
    backstory: |
      An expert organizer, skilled at turning scattered information into a structured book outline.
      You create clear, concise chapter outlines that guide the entire book-writing process.
    goal: "Generate a well-organized book outline with chapter titles and descriptions."
    tasks:
      generate_outline:
        description: |
          Create a book outline with chapters in sequential order based on the research findings.
          Ensure each chapter has a title and a brief description that highlights the topics and subtopics to be covered.
          Each chapter should be 3,000 words or less. Avoid duplicating chapters or topics in the outline.

          The goal of this book is to explore the metaphor of the Tree of Life, delving into its symbolism across cultures,
          its representation of growth, interconnectedness, and resilience, and how it relates to the human experience
          and natural world. The book aims to provide readers with insights into the Tree of Life as a powerful metaphor
          for understanding life's complexities, personal development, and interconnectedness.
        expected_output: "An outline of chapters, including titles and descriptions for each chapter's content."
    tools: []

  writer:
    role: "Chapter Writer"
    backstory: |
      A talented writer known for producing engaging, well-researched, and informative content.
      You excel at transforming complex ideas into readable and well-organized chapters in markdown format.
    goal: "Write well-structured chapters in markdown format, based on the provided chapter title and outline."
    tasks:
      write_chapter:
        description: |
          Write a well-structured chapter based on the chapter title, goal, and outline description.
          Each chapter should be written in markdown and should contain around 3,000 words.

          Here is the topic for the book: "The Metaphorical Tree of Life"
          Here is the title of the chapter: "Roots and Beginnings"

          **Chapter Description:**
          "An exploration of the roots as the foundation of the Tree of Life, symbolizing the origins of existence, connection to heritage, and the grounding principles that shape individual identity. This chapter delves into how roots anchor the tree and sustain growth, representing the importance of understanding one’s own beginnings and cultural heritage."

          Important notes:
          - The chapter you are writing needs to fit well with the other chapters in the book.
          - Ensure the content aligns with the overarching book structure.

          **Book Outline:**
          - **Chapter 1: Roots and Beginnings**
            An exploration of the roots as the foundation of the Tree of Life, symbolizing origins, connection to heritage, and grounding principles.

          - **Chapter 2: The Trunk – Stability and Growth**
            Examines the trunk as the support system of the tree, representing resilience, stability, and the journey of personal growth.

          - **Chapter 3: Branches of Knowledge**
            Discusses branches as pathways of exploration, illustrating how experiences, learning, and relationships expand one’s reach.

          - **Chapter 4: Leaves – Diversity and Change**
            Leaves symbolize individuality, diversity, and change, representing the unique qualities each person contributes to the world.

          - **Chapter 5: Fruits – Achievement and Legacy**
            Reflects on fruits as manifestations of effort and achievements, the legacy one leaves behind.

          - **Chapter 6: Seasons of Life**
            Analyzes the changing seasons as phases of life, symbolizing the cyclical nature of growth, dormancy, and renewal.
        expected_output: "A markdown-formatted chapter of around 3,000 words that covers the provided chapter title and outline."
    tools: []
dependencies: []
"""

## Main

In [52]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 10/10


 [2024-11-04 06:41:44][DEBUG]: == Working Agent: Research Agent
 [2024-11-04 06:41:44][INFO]: == Starting Task: Research the topic of "The Metaphorical Tree of Life" to gather the most important information that will 
be useful in creating a well-organized book outline. Focus on high-quality, reliable sources.

The goal of this book is to explore the metaphor of the Tree of Life, delving into its symbolism across cultures, 
its representation of growth, interconnectedness, and resilience, and how it relates to the human experience 
and natural world. The book aims to provide readers with insights into the Tree of Life as a powerful metaphor 
for understanding life's complexities, personal development, and interconnectedness.



> Entering new CrewAgentExecutor chain...
Action: [Search the internet]  
Action Input: {"search_query": "The Metaphorical Tree of Life symbolism across cultures"}  

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 


Search results: Title: TREE OF LIFE: SYMBOLISM ACROSS CULTURES - Mexicali Blues
Link: https://www.mexicaliblues.com/blogs/our-stories-mexicali-blues-blog/tree-of-life-symbolism-across-cultures-and-time?srsltid=AfmBOopD1UGamHYRrxKz56CwIUKhhZ8IxU4H0xwvPjn0dY26XsWcI9Ax
Snippet: the Tree of Life symbolizes many things— including wisdom, strength, protection, abundance, beauty, immortality, fertility and redemption ...
---
Title: Tree of Life Meaning Across Cultures | - Barbara Blake's Citizen Planet
Link: https://citizenplanet.com/?p=10826
Snippet: The Tree of Life also symbolizes immortality because even as the tree grows old, it creates seeds that carry its essence so it lives on through new saplings.
---
Title: The Tree of Life – Across Cultures | - Barbara Blake's Citizen Planet
Link: https://citizenplanet.com/?p=10851
Snippet: It is the tree that grows within the Garden of Eden and is the source of eternal life. There are several meanings behind the Tree of Life symbol in Christia

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 


Search results: Title: Tree of Life Meaning, Symbolism, and Influence - Saffron Marigold
Link: https://www.saffronmarigold.com/blog/tree-of-life-meaning-symbolism/?srsltid=AfmBOoqcDyZWuyxNoG0S2JM5xq3krwfhju4n7wsGVf7VpLlbXLbPQXhv
Snippet: The tree of life meaning includes enlightenment, wisdom, resilience, longevity, spiritual growth, and connectivity.
---
Title: The Tree of Life: A Symbol of Interconnectedness and Growth
Link: https://ystradgynlaisvc.org.uk/component/content/article/the-tree-of-life-a-symbol-of-interconnectedness-and-growth?catid=25&Itemid=101
Snippet: It is often seen in art, jewellery, and tattoos, signifying personal growth, strength, and a connection to one's roots.
---
Title: Spiritual Meaning of the Tree of Life | by Self Discovery 360 - Medium
Link: https://medium.com/@selfdiscovery360/spiritual-meaning-of-the-tree-of-life-b6fa3219a4bf
Snippet: The Tree of Life is an ancient and enduring symbol that transcends cultures, religions, and epochs. Representing gr

...

 


Search results: Title: Tree of Life Meaning, Symbolism, and Influence - Saffron Marigold
Link: https://www.saffronmarigold.com/blog/tree-of-life-meaning-symbolism/?srsltid=AfmBOooQA8IxPn86TanzoLaH9NcN822a0QemQodPH8BQvegLWlDJ-vok
Snippet: Its roots represent the divine connection to the earth and the ancestors, while its branches reach towards the heavens, symbolizing spiritual ...
---
Title: Discover the Tree of Life Meaning & Symbolism Across Cultures
Link: https://www.karmaandluck.com/blogs/home/tree-of-life-meaning?srsltid=AfmBOorfvTjxBadDv2JTfvj4YfpcItGDS0AJK0iO_2MWSTL17242oktx
Snippet: The roots, which represent the traditions and people who came before, keep the tree stable and balanced. The trunk usually represents the living individuals ...
---
Title: Honoring Your Irish Roots With The Tree Of Life - CladdaghRings.com
Link: https://www.claddaghrings.com/irish-roots-tree-of-life/
Snippet: The Tree of Life is the ideal symbol for Genealogy. ... Each element of the design spea

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 


Search results: Title: Tree of Life & Family Tree - Medium
Link: https://medium.com/@kerryjehanne/tree-of-life-family-tree-9184eaf3fbdb
Snippet: Culturally Constructed Meaning. It seems that much of our orientation to our life experiences is through the perspective of ascribed meaning.
---
Title: The Tree of Life Symbol Meaning - House Of Lor
Link: https://houseoflor.com/the-tree-of-life-symbol/
Snippet: Namely, the tree of life commonly symbolizes a connection to the afterlife, ancestral roots and divinity.
---
Title: The Celtic Tree of Life: Meaning and History - My Irish Jeweler
Link: https://www.myirishjeweler.com/blog/celtic-tree-of-life-meaning-history/?srsltid=AfmBOoqxDpkM0r5igqraI-Cm0-f08NMWUqxtUyrKXvzNX-EfmSUKu2np
Snippet: A Tree of Life can truly represent several qualities and feelings, the traditional meaning for a Celtic Tree of Life is Knowledge, Love, Health, and Prosperity.
---
Title: Tree of life - Wikipedia
Link: https://en.wikipedia.org/wiki/Tree_of_life
Snippet:

### Task Output ###
```markdown
# Chapter 1: Roots and Beginnings

## Introduction

The metaphorical Tree of Life serves as a profound symbol that resonates across cultures and disciplines, 
encapsulating the essence of existence, growth, and interconnectedness. Just as a tree's roots are crucial for its 
stability and nourishment, our roots—our origins, cultural heritage, and personal histories—form the foundation of 
who we are. This chapter, "Roots and Beginnings," explores how these roots anchor us, providing not only sustenance
but also a sense of belonging and identity. By understanding our beginnings, we can comprehend our place in the 
world and the intricate web of relationships that shape our lives.

## The Symbolism of Roots

### Roots as Foundations

Roots are often viewed as the unseen but vital components of a tree, grounding it and allowing it to flourish. 
Similarly, our roots—be they familial, cultural, or historical—establish a foundation upon which we build our 
identities. In many cultures, roots symbolize connection to ancestry and heritage, representing the values, 
traditions, and stories passed down through generations.

For instance, in Native American cultures, the concept of roots is deeply intertwined with the idea of 
interconnectedness with the earth and all living beings. The roots of a tree are seen as a metaphor for familial 
ties and the importance of honoring one’s ancestors, who provide not only genetic lineage but also cultural wisdom 
and guidance. This understanding reinforces the notion that to grow and thrive, one must acknowledge and respect 
their origins.

### Cultural Perspectives on Roots

Different cultures interpret the symbolism of roots in unique ways, reflecting their beliefs about life, creation, 
and existence. In ancient Egyptian mythology, for example, the Tree of Life was associated with the goddess Nut, 
who represented the sky and the cosmos. The roots of the tree were thought to connect to the underworld, 
illustrating the idea that life and death are intricately linked. This duality emphasizes that understanding our 
roots involves recognizing the cyclical nature of existence.

In contrast, the Celtic tradition celebrates the concept of the "World Tree," often depicted as an oak tree whose 
roots extend deep into the earth. This tree represents the interconnectedness of all life, with roots symbolizing 
the wisdom of the ancestors and the earth's nurturing energy. This metaphorical framework invites us to reflect on 
our own roots and how they inform our lives, choices, and values.

## The Role of Heritage in Shaping Identity

### Understanding Heritage

Heritage encompasses not only our family history but also the broader context of culture, language, and traditions 
that define us. It is a tapestry woven from the experiences, struggles, and triumphs of those who came before us. 
Embracing our heritage allows us to appreciate the cultural narratives that shape our identities and influence our 
perspectives.

For example, many individuals who immigrate to new countries often carry their cultural heritage with them, 
integrating it into their new lives while also adapting to their new environments. This dynamic process underscores
the importance of understanding one’s roots in navigating the complexities of identity formation.

### The Impact of Roots on Personal Identity

Our roots influence our values, beliefs, and worldviews. They shape our understanding of who we are and where we 
belong. A strong connection to one’s heritage can instill a sense of pride and purpose, while a disconnection from 
one’s roots may lead to feelings of alienation or confusion about identity.

Consider the example of individuals who have been adopted or raised in different cultural contexts. They often face
the challenge of reconciling their adoptive identities with their biological roots. This journey of self-discovery 
emphasizes the importance of exploring one's origins as a means of c

None
